### WINDOW FUNCITON

In [ ]:
# accumulated sum
SELECT SUM(weight) OVER(ORDER BY turn) FROM Queue

In [ ]:
# accumulated sum by month- get sum for each month first-then subquery
SELECT t.Month, SUM(t.Qty) OVER(ORDER BY Month) FROM
    (SELECT MONTH(join_date) AS Month, COUNT(DISTINCT driver_id) AS Qty FROM Drivers
    GROUP BY MONTH(join_date))t

In [ ]:
# accumulate sum for each player, at that event date
SELECT player_id, 
event_date, 
SUM(games_played) OVER(PARTITION BY player_id ORDER BY event_date) AS games_played_so_far FROM Activity 

### RANK OVER

In [ ]:
SELECT *,DENSE_RANK() OVER(ORDER BY Salary DESC) AS rank FROM Employee
#rank this column doesn't come with the table at the same time, which means:
#can not use where 'rank'=1 directly below this query

# 可以先rank，在select subset of the columns
SELECT score, DENSE_RANK() OVER(ORDER BY Score DESC) AS Rank FROM Scores


#MySQL
# don't name a column using sensitive word, i.e: rank, instead using d_rank, r, ...
SELECT DISTINCT t.Salary AS getNthHighestSalary FROM
  (SELECT *, DENSE_RANK() OVER(ORDER BY Salary DESC) AS d_rank FROM Employee) AS t
WHERE t.d_rank = N

In [ ]:
SELECT customer_id, product_id, DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(order_id) DESC) AS Rank FROM Orders
    GROUP BY customer_id, product_id

In [ ]:
DENSE_RANK() OVER(ORDER BY Salary DESC) #111 2
RANK() OVER(ORDER BY Salary DESC) #111 4
ROW_NUMBER() OVER(ORDER BY Salary DESC) #1234

In [ ]:
PERCENT_RANK() OVER(ORDER BY )

In [ ]:
#PARTITION BY
SELECT e.Name AS Employee, e.Salary AS Salary,d.Name AS Department, 
 DENSE_RANK() OVER(PARTITION BY e.DepartmentId ORDER BY e.Salary DESC) AS rank FROM Employee e

In [ ]:
# agg function in partition by 

WITH CTE AS
    (SELECT*FROM
        (SELECT customer_id, product_id, DENSE_RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(customer_id) DESC) AS Rank FROM Orders
        GROUP BY customer_id, product_id)t
    WHERE t.Rank = 1
    )

In [ ]:
# multiple window function/WHERE statment in same query

SELECT* FROM Student
WHERE student_id NOT IN
    (SELECT DISTINCT student_id FROM 
        (SELECT*,
        DENSE_RANK() OVER(PARTITION BY exam_id ORDER BY score DESC) AS rank_high,
        DENSE_RANK() OVER(PARTITION BY exam_id ORDER BY score) AS rank_low
        FROM Exam)t
    WHERE t.rank_high = 1 OR t.rank_low = 1)
AND student_id IN (SELECT DISTINCT student_id FROM Exam)

In [ ]:
# using MAX func to return NULL instead of nothing after a query

# query 1
SELECT MAX(t.Salary) AS SecondHighestSalary FROM 
    (SELECT*, DENSE_RANK() OVER(ORDER BY Salary DESC) AS rk FROM Employee)t
WHERE t.rk = 2

# query 2
SELECT t.Salary AS SecondHighestSalary FROM 
    (SELECT*, DENSE_RANK() OVER(ORDER BY Salary DESC) AS rk FROM Employee)t
WHERE t.rk = 2

### Having vs Where clause

In [ ]:
# having is against(and used after) aggregate function, e.g. sum
# where is filering on raw data

# where-groupby-having
SELECT s.course, AVERAGE(s.score) as avg FROM Student s
WHERE s.age > 18
GROUP BY s.course
HAVING avg < 60

# having average(score)>90

### CASE WHEN

In [ ]:
SELECT
CASE
    WHEN id%2=0 THEN id-1
    WHEN id=(SELECT COUNT(*) FROM seat) THEN id
    ELSE id+1
END id #只对id 这一列操作
,student FROM seat
ORDER BY id
#generate the column when searching at the same time

In [ ]:
SELECT 
CASE 
    WHEN A=B AND B=C THEN 'Equilateral'
    WHEN A+B<=C OR B+C<=A OR A+C<=B THEN 'Not A Triangle'
    WHEN A+B>C AND B+C>A AND A+C>B AND A!=B AND B!=C AND C!=A THEN 'Scalene'
    ELSE 'Isosceles'
END AS Result
FROM TRIANGLES 

### CHANGE WITH DATE FORMAT(2019/9/12)

In [ ]:
#USE DATEDIFF()
SELECT w1.Id FROM Weather w1 LEFT JOIN Weather w2 
ON DATEDIFF(Day,w2.RecordDate,w1.RecordDate)=1 #w1-w2
WHERE w1.Temperature>w2.Temperature

### SELECT TABLE ALIAS

In [2]:
#Format
#SELECT T1.Name FROM (SELECT NAME,RANK()OVER(ORDER BY SCORE DESC) rank FROM TABLE1) AS T1

In [ ]:
SELECT * FROM
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Doctor') AS d,
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Professor') AS p

### CTE: WITH AS ...

In [ ]:
# create a temp table
# be able to do later manipulation on that table

#FORMAT
WITH NewTableOne AS(
SELCT* FROM t
WHERE t.score > 60
    ...
)
SELECT * FROM NewTableOne t1
INNER JOIN NewTableOne t2
ON t1.timestamp = t2.timestamp AND t1.RackNo+t2.Rack = 3

In [ ]:
# two CTE statements
WITH CTE_1 AS
    (SELECT COUNT(DISTINCT(CONCAT(sender_id, send_to_id))) AS fr FROM FriendRequest),
    CTE_2 AS 
    (SELECT COUNT(DISTINCT(CONCAT(requester_id, accepter_id))) AS ra FROM RequestAccepted)

In [ ]:
# recursive CTE-generate a month table
WITH CTE AS
(
    SELECT 1 AS Month
    UNION ALL  # can only use UNION ALL but not UNION
    SELECT Month+1 FROM CTE
    WHERE Month < 12
)

In [ ]:
# mysql
WITH RECURSIVE MONTH_CTE AS(
    SELECT 1 AS Month 
    UNION ALL
    SELECT Month+1 FROM MONTH_CTE
    WHERE Month < 12
),

In [ ]:
WITH Month_CTE AS
(
    SELECT 1 AS Month
    UNION ALL
    SELECT Month+1 FROM Month_CTE
    WHERE Month < 12
),

CTE_1 AS(
    SELECT t2.Month, COUNT(DISTINCT t2.driver_id) AS active_drivers FROM
        (SELECT* FROM Month_CTE m
        LEFT JOIN(
            SELECT 
            CASE 
                WHEN YEAR(join_date)=2019 THEN 1
                ELSE MONTH(join_date)
            End AS Month_d, driver_id
            FROM Drivers d 
            WHERE YEAR(join_date) < 2021
        )d ON m.Month >= d.Month_d) t2
    GROUP BY t2.Month
),

CTE_2 AS(
    SELECT t3.Month_r, COUNT(DISTINCT(t3.ride_id)) AS accepted_rides FROM
        (SELECT
        CASE
            WHEN YEAR(requested_at)= 2019 THEN 1
            ELSE MONTH(requested_at)
        END AS Month_r, ride_id AS ride_id
        FROM AcceptedRides a
        LEFT JOIN Rides r ON a.ride_id = r.ride_id
        WHERE YEAR(r.requested_at) < 2021) t3
    GROUP BY t3.Month_r
)

SELECT CTE_1.Month,CTE_1.active_drivers, ISNULL(CTE_2.accepted_rides,0) FROM CTE_1
LEFT JOIN CTE_2 ON CTE_1.Month = CTE_2.Month_r

### ORDER BY

In [ ]:
# ORDER BY aggregation function
SELECT TOP 1 customer_number FROM Orders
GROUP BY customer_number
ORDER BY COUNT(order_number) DESC

### GROUP BY

In [ ]:
# GROUP BY not necessiary to call- group by column, can just call agg columns
SELECT MIN(t.log_id) AS start_id, MAX(t.log_id) AS end_id FROM
    (SELECT *, ROW_NUMBER() OVER(ORDER BY log_id ) AS row_no FROM Logs)t
GROUP BY t.log_id - t.row_no

### JOIN

In [ ]:
SELECT* FROM Activity a1
INNER JOIN Activity a2 
ON a1.machine_id = a2.machine_id AND a1.process_id = a2.process_id
AND a1.activity_type = 'end' AND a2.activity_type = 'start'

In [ ]:
# use less subquery!
SELECT a1.machine_id, ROUND(AVG(a1.timestamp-a2.timestamp),3) AS processing_time FROM Activity a1
INNER JOIN Activity a2 
ON a1.machine_id = a2.machine_id AND a1.process_id = a2.process_id
AND a1.activity_type = 'end' AND a2.activity_type = 'start'
GROUP BY a1.machine_id

### MULTPLE JOIN

In [ ]:
#Join consecutively
SELECT*FROM a
JOIN b ON a.Id=b.Id 
Join c ON b.Id=c.Id
a--b--c
reference:https://www.interfacett.com/blogs/multiple-joins-work-just-like-single-joins/

In [ ]:
SELECT d.Name,p.Name,s.Name,a.Name FROM
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Doctor') AS d
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Professor') AS p
ON d.Row=p.Row
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Singer') AS s
ON p.Row=s.Row
FULL OUTER JOIN 
(SELECT Name,ROW_NUMBER() OVER(ORDER BY Name) AS Row FROM OCCUPATIONS
WHERE Occupation='Actor') AS a
ON s.Row=a.Row

### JOIN BY (i1,i2)

In [ ]:
SELECT
ROUND(MIN(SQRT(POWER((p1.x-p2.x),2)+POWER((p1.y-p2.y), 2))), 2) AS shortest
FROM point_2d p1
INNER JOIN point_2d p2 ON (p1.x,p1.y) != (p2.x,p2.y)

### Calculate percentage%

In [ ]:
SELECT
t.Request_at AS Day,
ROUND(SUM(CASE WHEN t.Status Like 'cancelled%' Then 1 ELSE 0 END)/COUNT(t.id), 2)  AS Cancellation_Rate
FROM Trips t
INNER JOIN Users uc ON t.Client_Id = uc.Users_Id
INNER JOIN Users ud ON t.Driver_Id = ud.Users_Id
WHERE uc.Banned = 'No' AND ud.Banned = 'No' AND t.Request_at Between '2013-10-01' AND '2013-10-03'
GROUP BY t.Request_at

### TOP1

In [ ]:
SELECT TOP 1 requester_id AS id, COUNT(accepter_id) AS num FROM
    (SELECT* FROM request_accepted
    UNION
    SELECT accepter_id, requester_id, accept_date FROM request_accepted) t
GROUP BY requester_id
ORDER BY num DESC

### Between and 

In [ ]:
# small note for GROUPBY!!!
# we can:
SELECT DISTINCT col_a FROM
GROUP BY col_a, col_b
HAVING COUNT(DISTINCT col_b) >= 4

In [ ]:
# https://docs.microsoft.com/en-us/sql/t-sql/language-elements/exists-transact-sql?view=sql-server-ver15

SELECT DISTINCT l1.id FROM Logins l1
    INNER JOIN Logins l2 ON l1.id = l2.id AND DATEDIFF(day, l1.login_date, l2.login_date) BETWEEN 1 AND 4
    GROUP BY l1.id, l1.login_date
    HAVING COUNT(DISTINCT l2.login_date) = 4

### Union/Union all/INTERCEPT/EXCEPT

In [ ]:
SELECT DISTINCT customer_id FROM Orders 
WHERE product_name = 'A' 
INTERSECT
SELECT DISTINCT customer_id FROM Orders 
WHERE product_name = 'B' 
EXCEPT 
SELECT DISTINCT customer_id FROM Orders
WHERE product_name = 'C'

### PERCEDING

In [ ]:
# SUM previous 10(n) rows to current row--rolling
#  i will have sum(index-3, index-2, index-1) in current row

SUM(DC_CURRENT) OVER(PARTITION BY "RackNo"
                    ORDER BY "Timestamp"
                    ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING) AS "DC_CURRENT_SUM_3"

### ISNULL

In [ ]:
ISNULL(CTE_2.accepted_rides,0)

### MAX-MIN

In [ ]:
SELECT (MAX(POPULATION)-MIN(POPULATION)) a FROM CITY#can be done in one step

### MEDIAN

In [ ]:
SELECT DISTINCT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY LAT_N) OVER () FROM STATION 
#if you want to get percentile in a subgroup you can do partition by in over clause

### REMOVE 0 IN NUMBERS

In [ ]:
SELECT DISTINCT
CEILING(AVG(Salary)-
AVG(CAST(REPLACE(STR(Salary),'0','') AS INT)) )+1
FROM EMPLOYEES
#CEILING ROUND TO LOWEST INTERGER

### TIMESTAMP

In [ ]:
#SELECT id FROM TABLE WHERE YEAR(FROM_UNIXTIME(creation_date)) = 2010
YEAR(CAST(timestamp AS DATE))=2018

### MULTIPLY 1 COLUMN TO ANOTHER& SUBQUERY FROM A TEMPORARY TABLE

In [ ]:
SELECT* FROM
(SELECT (months*salary) AS earnings,DENSE_RANK() OVER(ORDER BY months*salary DESC) rank FROM Employee) AS T

### ROUNDS TO 2 DECIMAL

In [ ]:
33.3333
# DECIMAL -33.00
# ROUND 2 - 33.33

SELECT CONVERT(DECIMAL(10,2),SUM(LAT_N)) FROM table1#123.45

ROUND(AVG(c1.POPULATION),1)#round to nearest integer

### SELECT TWO DIGITS OPERATION

In [ ]:
SELECT ROUND(100.0* t1.immediate_delivery/ t2.all_delivery, 2) AS immediate_percentage FROM 
    (SELECT COUNT(delivery_id) AS immediate_delivery FROM Delivery
    WHERE order_date = customer_pref_delivery_date) t1,
    (SELECT COUNT(delivery_id) AS all_delivery FROM Delivery) t2

### TOP 10 PERCENT

In [ ]:
SELECT NTILE(10) OVER(ORDER BY ID) FROM STATION 

In [ ]:
### UPPER/LOWERCASE
CONCAT(UPPER(SUBSTRING(name,1,1)),LOWER(SUBSTRING(name,2,LENGTH(NAME))))